In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re
import os

In [ ]:
def get_sorted_csv_files_std(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_std_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    # print(sorted_csv_files)
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def read_csv_files(directory, file_list):
    data_frames = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    return data_frames

In [ ]:
def get_sorted_csv_files(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    # print(sorted_csv_files)
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def read_csv_files(directory, file_list):
    data_frames = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    return data_frames

In [ ]:
network_name = "result_network10_4"
seed="seed0"
system_name = "BO_system_CT"
directory_path=os.path.join(network_name, system_name,"true_estimate",seed)

sorted_file_list_std = get_sorted_csv_files_std(directory_path)
data_frame_list_std = read_csv_files(directory_path, sorted_file_list_std)



sorted_file_list = get_sorted_csv_files(directory_path)
data_frame_list = read_csv_files(directory_path, sorted_file_list)

# 獲得関数のdfリストを作成
alpha=3.0
data_frame_list_act = []
for i in range(len(data_frame_list)):
    df = data_frame_list[i]
    df_std = data_frame_list_std[i]
    df_act = pd.DataFrame({
        "true": df["true"],
        "estimate": df["estimate"] + alpha * df_std["estimate"],
        "mat_index": df["mat_index"]
    })
    data_frame_list_act.append(df_act)
    tmp = df["estimate"].to_numpy() + alpha * df_std["estimate"].to_numpy()
print(data_frame_list_act)

In [ ]:
def plot_true_estimate(data_frame_list,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    for idx, df in enumerate(data_frame_list):
        color = cmap(idx / len(data_frame_list))
        # plt.scatter(df['true'], df['estimate'], label=f'File {idx}', color=color)
        plt.scatter(df['true'], df['estimate'], color=color,marker=".",alpha=0.1)
        arg = df["estimate"].argmax()
        plt.scatter(df['true'][arg], df['estimate'][arg], color="r",marker=".",alpha=1.0)
        # print(df['true'][arg], df['estimate'][arg])
    
    # y=x の灰色の直線をプロット
    min_val = min(df['true'].min() for df in data_frame_list)
    max_val = max(df['true'].max() for df in data_frame_list)
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--')
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('True')
    plt.ylabel('Estimate')
    plt.title(title)
    # plt.legend()
    plt.show()

In [ ]:
# plot_true_estimate(data_frame_list_std, title='True vs Estimate CT')
plot_true_estimate(data_frame_list, title='True vs Estimate MM')

In [ ]:
# 予測値と分散値で再計算した獲得関数による探索候補点
for i in range(len(data_frame_list_act)):
    arg = data_frame_list_act[i]["estimate"].argmax()
    print(data_frame_list_act[i]["true"][arg])

In [ ]:
# 絶対インデックスを追跡することによる、探索候補点
# index_list = []
# for i in range(len(data_frame_list_act)-1):
#     print(len(data_frame_list_act[i]["mat_index"]),len(data_frame_list_act[i+1]["mat_index"]))
#     diff = set(data_frame_list_act[i]["mat_index"].to_numpy()).difference(set(data_frame_list_act[i+1]["mat_index"].to_numpy()))
#     print(list(data_frame_list_act[i]["true"][data_frame_list_act[i]["mat_index"]==list(diff)[0]])[0])
#     index_list.append(list(diff)[0])

#絶対インデックスが正しいか確認してみる
# df = pd.read_csv("network10_4/BO_system_CT/complete_B2_target.csv",header=None)
# print(df[0][index_list])

In [ ]:
# データセットに重複がある問題

# df = pd.read_csv("network10_4/BO_system_CT/complete_input_features.csv",header=None)
# composition_list = []
# for i in range(len(df)):
#     composition_list.append(tuple(df.iloc[i]))
# print(len(composition_list),len(set(composition_list)))

# # どの要素がどれと重複しているのか、グループ化してみる
# composition_dict = {}
# for i in range(len(df)):
#     composition = tuple(df.iloc[i])
#     if composition not in composition_dict:
#         composition_dict[composition] = []
#     composition_dict[composition].append(i)

# # 重複している要素を表示
# composition_dict_duplicates = {k: v for k, v in composition_dict.items() if len(v) > 1}
# # for composition, indices in composition_dict_duplicates.items():
# #     print(f"Composition: {composition}, Indices: {indices}")

# df2 = pd.read_csv("network10_4/BO_system_CT/complete_B2_target.csv",header=None)
# for composition, indices in composition_dict_duplicates.items():
#     print(df2.iloc[indices])

In [ ]:
def plot_true_estimate_one(data_frame_list,index=0,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    # for idx, df in enumerate(data_frame_list):
    idx=index
    df=data_frame_list[idx]
    color = cmap(idx / len(data_frame_list))
    plt.scatter(df['true'], df['estimate'], color=color,marker=".",alpha=0.5)
    
    # y=x の灰色の直線をプロット
    min_val = min(df['true'].min() for df in data_frame_list)
    max_val = max(df['true'].max() for df in data_frame_list)
    plt.plot([min_val, max_val], [min_val, max_val], color='gray', linestyle='--')
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('True')
    plt.ylabel('Estimate')
    plt.title(title)
    # plt.legend()
    plt.show()

In [ ]:
plot_true_estimate_one(data_frame_list, 0, title='True vs Estimate CT(0)')
plot_true_estimate_one(data_frame_list, 99, title='True vs Estimate CT(99)')
# plot_true_estimate_one(data_frame_list, 199, title='True vs Estimate CT(199)')

In [ ]:
def plot_true_histgram(data_frame_list,title='True vs Estimate'):
    # カラーマップを取得
    cmap = cm.get_cmap('viridis', len(data_frame_list))
    
    plt.figure(figsize=(7, 6))
    
    for idx, df in enumerate(data_frame_list):
        color = cmap(idx / len(data_frame_list))
        # plt.scatter(df['true'], df['estimate'], label=f'File {idx}', color=color)
        plt.hist(df['true'], bins=100,color=color,alpha=0.5,log=True)
    
    
    # カラーバーを追加
    norm = plt.Normalize(0, len(data_frame_list))
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    plt.colorbar(sm)


    plt.xlabel('Curie Temperature')
    plt.ylabel('Count')
    plt.title(title)
    # plt.legend()
    plt.show()

def plot_histogram_difference(data_frame_list, title='Histogram Difference'):
    if len(data_frame_list) < 2:
        raise ValueError("data_frame_list must contain at least two data frames.")
    
    # 最初と最後のデータフレームを取得
    df_first = data_frame_list[0]
    df_last = data_frame_list[-1]
    
    # ヒストグラムを計算
    hist_first, bins_first = np.histogram(df_first['true'], bins=100)
    hist_last, bins_last = np.histogram(df_last['true'], bins=bins_first)
    
    # 差分を計算
    hist_diff =  hist_first - hist_last
    print(hist_diff)
    print(sum(hist_diff))
    
    # プロット
    plt.figure(figsize=(7, 6))
    plt.hist(bins_first[:-1], bins_first, weights=hist_diff, alpha=0.5)
    
    plt.xlabel('True')
    plt.ylabel('Difference in Count')
    plt.title(title)
    plt.show()

In [ ]:
plot_true_histgram(data_frame_list, title='True histgram CT')

In [ ]:
plot_histogram_difference(data_frame_list, title='Histogram searched CT')

In [ ]:
def calculate_mae(data_frame_list):
    mae_list = []
    for df in data_frame_list:
        mae = np.mean(np.abs(df['true'] - df['estimate']))
        # estimateの分布を整える。estimateの最小値が0になるように調整し算出する。
        # mae = np.mean(np.abs(df['true'] - (df['estimate'] - df['estimate'].min())))
        mae_list.append(mae)
    return mae_list

def plot_mae(mae_list):
    plt.figure(figsize=(10, 6))
    plt.plot(mae_list, marker='o', linestyle='-', color='b')
    plt.xlabel('File Index')
    plt.ylabel('MAE')
    plt.title('Mean Absolute Error (MAE) for Each File')
    plt.show()

In [ ]:
# MAEを計算
mae_list = calculate_mae(data_frame_list)
# MAEをプロット
plot_mae(mae_list)

trainとvalのデータを取得し、normalizationに使った平均と分散を求める

NNの出力を逆算する。

In [ ]:
def get_sorted_csv_files_train(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_train_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def get_sorted_csv_files_test(directory):
    # 正規表現パターンを定義
    pattern = re.compile(r'true_estimate_test_(\d{4})\.csv')
    
    # ディレクトリ内のファイルを取得
    files = os.listdir(directory)
    
    # ファイル名と4桁の数字を抽出
    csv_files = []
    for file in files:
        match = pattern.match(file)
        if match:
            csv_files.append((file, int(match.group(1))))
    
    # 4桁の数字でソート
    sorted_csv_files = sorted(csv_files, key=lambda x: x[1])
    
    # ソートされたファイル名のリストを取得
    sorted_file_names = [file[0] for file in sorted_csv_files]
    
    return sorted_file_names

def read_csv_files(directory, file_list):
    data_frames = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    return data_frames

In [ ]:
sorted_file_list_train = get_sorted_csv_files_train(directory_path)
sorted_file_list_test = get_sorted_csv_files_test(directory_path)
data_frame_list_train = read_csv_files(directory_path, sorted_file_list_train)
data_frame_list_test = read_csv_files(directory_path, sorted_file_list_test)

In [ ]:
# 各探索時のnormalizationに利用した平均と分散を算出

mean_list = []
std_list = []
for i in range(len(data_frame_list_train)):
    cocat_observed_true = pd.concat([data_frame_list_train[i]["observed"], data_frame_list_test[i]["observed"]])
    mean_list.append(cocat_observed_true.mean())
    std_list.append(cocat_observed_true.std())

In [ ]:
plt.plot(mean_list)

In [ ]:
plt.plot(std_list)

In [ ]:
# unobservedのestimateについて、normalizationを行う
data_frame_list_new = data_frame_list.copy()

mean = 48.843473
std = 102.022205
for i in range(len(data_frame_list_new)):
    data_frame_list_new[i]["estimate"] = (data_frame_list_new[i]["estimate"] - mean) / std

# unobsdervedのestimateについて、正しい値でdenormalizationを行う
for i in range(len(data_frame_list_new)):
    data_frame_list_new[i]["estimate"] = data_frame_list_new[i]["estimate"] * std_list[i] + mean_list[i]


In [ ]:
plot_true_estimate(data_frame_list_new, title='True vs Estimate CT(fixed)')

In [ ]:
plot_true_estimate_one(data_frame_list_new, 0, title='True vs Estimate CT(0)')
# plot_true_estimate_one(data_frame_list_new, 99, title='True vs Estimate CT(99)')

In [ ]:
# MAEを計算
mae_list_new = calculate_mae(data_frame_list_new)
# MAEをプロット
plot_mae(mae_list_new)